In [1]:
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd

seasons = ["2023-24", "2024-25"]
dfs = []

for season in seasons:
    gamefinder = leaguegamefinder.LeagueGameFinder(
        season_nullable=season,
        season_type_nullable="Regular Season"
    )
    df = gamefinder.get_data_frames()[0]

    dfs.append(df)

league_games = pd.concat(dfs, ignore_index = True)

#No Duplicates
league_games = league_games.drop_duplicates(subset=["GAME_ID"])
DroppedNAN = league_games.dropna(subset=['PTS', 'AST', 'REB', 'TOV', 'PLUS_MINUS', 'FG_PCT', 'FT_PCT', 'FG3_PCT', 'WL'])


DroppedNAN.to_csv("NBA_Dataset.csv", index = False)


In [2]:
uiHomeTeam = input("Select the 1st NBA team: ")
uiAwayTeam = input("Select the 2nd NBA team: ")

df = pd.read_csv("NBA_Dataset.csv")
df = pd.DataFrame(df)

# Convert W/L to 1/0
df["WL"] = df["WL"].map({"W": 1, "L": 0}).astype(int)

FEATURES = ["PTS","REB","AST","TOV","PLUS_MINUS","FG_PCT","FG3_PCT","FT_PCT"]
X = df[FEATURES]
y = df["WL"].values

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# Fit MiniBatchKMeans on TRAIN only
k = 6
mbk = MiniBatchKMeans(n_clusters=k, batch_size=256, random_state=42)
mbk.fit(X_train)

# Cluster assignments
train_clusters = mbk.predict(X_train)
test_clusters  = mbk.predict(X_test)

train_cluster_win = pd.Series(y_train).groupby(train_clusters).mean()

global_win = float(np.mean(y_train))
cluster_win_prob = np.array([float(train_cluster_win.get(i, global_win)) for i in range(k)])

df["Cluster"] = mbk.predict(X_scaled)


In [3]:
# Team-level feature averages
team_averages = df.groupby("TEAM_NAME")[FEATURES].mean()


In [4]:
from scipy.special import softmax

def get_last_n_games(team_name, n=5):
    team_games = df[df["TEAM_NAME"] == team_name].tail(n)
    return team_games[FEATURES].mean().values

def raw_win_probability(stats):
    # Convert a stats vector into a win-probability using cluster distances.
    scaled = scaler.transform([stats])
    distances = mbk.transform(scaled)[0]          # shape: (k,)
    weights = softmax(-distances)                # closer clusters get higher weight
    return float(weights @ cluster_win_prob)      # weighted avg of per-cluster win rates

def normalize_probs(pA, pB):
    total = pA + pB
    return pA / total, pB / total

def predict_future_game(teamA_name, teamB_name, n=5):
    teamA_stats = get_last_n_games(teamA_name, n)
    teamB_stats = get_last_n_games(teamB_name, n)

    rawA = raw_win_probability(teamA_stats)
    rawB = raw_win_probability(teamB_stats)

    probA, probB = normalize_probs(rawA, rawB)
    winner = teamA_name if probA > probB else teamB_name

    print("🏀 Future Game Prediction")
    print("Predicted Winner:", winner)
    print(f"{teamA_name} Win Probability: {probA:.2%}")
    print(f"{teamB_name} Win Probability: {probB:.2%}")

    return winner, probA, probB


In [5]:
from scipy.special import softmax

def kmeans_win_proba(Xs_scaled):
    distances = mbk.transform(Xs_scaled)                 # (n, k)
    weights = softmax(-distances, axis=1)               # row-wise
    return (weights @ cluster_win_prob).astype(float)   # (n,)

# Add per-game predicted probability/label for the full dataset
p_all = kmeans_win_proba(X_scaled)
df["Predicted_Prob"] = p_all
df["Predicted_WL"] = (p_all >= 0.5).astype(int)

accuracy = (df["Predicted_WL"] == df["WL"]).mean()
print(f"Model Accuracy (full data): {accuracy:.2%}")

# Holdout outputs for evaluation in the next cell
p_test = kmeans_win_proba(X_test)


Model Accuracy (full data): 77.51%


In [ ]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, brier_score_loss

import numpy as np

def eval_prob_model(name, y_true, p):
    """Evaluation for a model that outputs win probabilities (supervised metrics)."""
    y_true = np.asarray(y_true).astype(int)
    p = np.asarray(p).astype(float)

    pred = (p >= 0.5).astype(int)

    print(f"\n{name}")
    print("  logloss :", log_loss(y_true, p))
    print("  brier   :", brier_score_loss(y_true, p))

    # ROC AUC needs both classes present
    if len(np.unique(y_true)) > 1:
        print("  auc     :", roc_auc_score(y_true, p))
    else:
        print("  auc     : N/A (single class in y_true)")

    print("  acc     :", accuracy_score(y_true, pred))

# ---- Evaluations ----
test_clusters = mbk.predict(X_test)

eval_prob_model("KMEANS-PROB TEST", y_test, p_test)

# ---- Predict the user-selected matchup ----
predict_future_game(uiHomeTeam, uiAwayTeam)



KMEANS-PROB TEST
  logloss : 0.5119480515438987
  brier   : 0.16508333798372476
  auc     : 0.8949344270428499
  acc     : 0.8048780487804879
🏀 Future Game Prediction
Predicted Winner: New York Knicks
New York Knicks Win Probability: 63.63%
San Antonio Spurs Win Probability: 36.37%


C:\Users\denni\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\denni\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


('New York Knicks', 0.6362923793383006, 0.36370762066169937)